In [2]:
import librosa
import numpy as np
import pandas as pd
from scipy.stats import entropy
from scipy.signal import hilbert
import nolds
from pathlib import Path

def safe_entropy(data, bins=10):
    hist = np.histogram(data, bins=bins)[0]
    hist = hist[hist > 0]
    return entropy(hist) if len(hist) > 1 else 0

def extract_features(file_path, status):
    try:
        y, sr = librosa.load(file_path, sr=None)
        features = {'name': file_path.stem, 'status': status}
        
        # 1. Основные частотные характеристики
        f0 = librosa.yin(y, fmin=50, fmax=500)
        f0 = f0[f0 > 0]  # Убираем нулевые значения
        
        features.update({
            'MDVP:Fo(Hz)': np.mean(f0) if len(f0) > 0 else 0,
            'MDVP:Fhi(Hz)': np.max(f0) if len(f0) > 0 else 0,
            'MDVP:Flo(Hz)': np.min(f0) if len(f0) > 0 else 0,
        })

        # 2. Jitter и его варианты
        if len(f0) > 1:
            diffs = np.diff(f0)
            abs_diffs = np.abs(diffs)
            jitter_percent = np.mean(abs_diffs) / np.mean(f0) * 100
            jitter_abs = np.mean(abs_diffs)
            
            # RAP (Relative Average Perturbation)
            rap = np.mean(np.abs(diffs[:-1] + diffs[1:]) / 2) / np.mean(f0) * 100
            
            # PPQ (Five-point Period Perturbation Quotient)
            if len(f0) > 4:
                ppq = np.mean([np.abs(f0[i] - (f0[i-2] + f0[i-1] + f0[i+1] + f0[i+2]) / 4) 
                            for i in range(2, len(f0)-2)]) / np.mean(f0) * 100
            else:
                ppq = 0
                
            # DDP (Jitter:DDP)
            ddp = rap * 3
        else:
            jitter_percent = jitter_abs = rap = ppq = ddp = 0

        features.update({
            'MDVP:Jitter(%)': jitter_percent,
            'MDVP:Jitter(Abs)': jitter_abs,
            'MDVP:RAP': rap,
            'MDVP:PPQ': ppq,
            'Jitter:DDP': ddp,
        })

        # 3. Shimmer и его варианты
        amplitude = np.abs(hilbert(y))
        if len(amplitude) > 1:
            amp_diffs = np.abs(np.diff(amplitude))
            shimmer = np.mean(amp_diffs) / np.mean(amplitude) * 100
            shimmer_db = 20 * np.log10((np.mean(amp_diffs) + 1e-10) / np.mean(amplitude))
            
            # APQ3 (3-point Amplitude Perturbation Quotient)
            if len(amplitude) > 3:
                apq3 = np.mean([np.abs(amplitude[i] - (amplitude[i-1] + amplitude[i] + amplitude[i+1]) / 3)
                               for i in range(1, len(amplitude)-1)]) / np.mean(amplitude) * 100
            else:
                apq3 = 0
                
            # APQ5 (5-point Amplitude Perturbation Quotient)
            if len(amplitude) > 5:
                apq5 = np.mean([np.abs(amplitude[i] - (amplitude[i-2] + amplitude[i-1] + amplitude[i] + 
                                amplitude[i+1] + amplitude[i+2]) / 5) for i in range(2, len(amplitude)-2)]) / np.mean(amplitude) * 100
            else:
                apq5 = 0
                
            # DDA (Shimmer:DDA)
            dda = apq3 * 3
        else:
            shimmer = shimmer_db = apq3 = apq5 = dda = 0

        features.update({
            'MDVP:Shimmer': shimmer,
            'MDVP:Shimmer(dB)': shimmer_db,
            'Shimmer:APQ3': apq3,
            'Shimmer:APQ5': apq5,
            'MDVP:APQ': apq5,  # Обычно APQ = APQ5
            'Shimmer:DDA': dda,
        })

        # 4. Шумовые характеристики
        harmonic, percussive = librosa.effects.hpss(y)
        features['NHR'] = np.sum(percussive**2) / (np.sum(harmonic**2) + 1e-10)
        features['HNR'] = 10 * np.log10(np.sum(harmonic**2) / (np.sum(percussive**2) + 1e-10))

        # 5. Нелинейные характеристики
        features['RPDE'] = safe_entropy(f0) if len(f0) > 0 else 0
        features['DFA'] = nolds.dfa(y) if len(y) > 100 else 0
        
        # Для spread1, spread2 используем упрощенные метрики
        if len(f0) > 1:
            features['spread1'] = np.std(f0) / np.mean(f0)
            features['spread2'] = np.quantile(f0, 0.9) - np.quantile(f0, 0.1)
        else:
            features['spread1'] = features['spread2'] = 0
            
        # Заменяем D2 на альтернативную метрику (например, sample entropy)
        features['D2'] = 0  # Заглушка, так как correlation_dimension недоступна
        
        features['PPE'] = safe_entropy(np.diff(f0)) if len(f0) > 1 else 0

        return features
    
    except Exception as e:
        print(f"Ошибка при обработке файла {file_path}: {str(e)}")
        return None

# Обработка файлов
dataset = []

# Папка с пациентами (статус 1)
pd_dir = Path("processed_audio_segments_test\PD_AH")
for audio_file in pd_dir.glob("*.wav"):
    features = extract_features(audio_file, status=1)
    if features is not None:
        dataset.append(features)

# Папка со здоровыми (статус 0)
hc_dir = Path("processed_audio_segments_test\HC_AH")
for audio_file in hc_dir.glob("*.wav"):
    features = extract_features(audio_file, status=0)
    if features is not None:
        dataset.append(features)

# Сохранение в CSV
if dataset:
    df = pd.DataFrame(dataset)
    # Убедимся, что все колонки присутствуют (на случай, если какие-то файлы не обработались)
    expected_columns = ['name', 'MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 
                       'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 
                       'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 
                       'Shimmer:APQ5', 'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 
                       'RPDE', 'DFA', 'spread1', 'spread2', 'D2', 'PPE']
    
    # Добавляем отсутствующие колонки с нулевыми значениями
    for col in expected_columns:
        if col not in df.columns:
            df[col] = 0
    
    df = df[expected_columns]  # Приводим к правильному порядку колонок
    df.to_csv("parkinson_test_dataset.csv", index=False)
    print(f"Датасет сохранен как 'parkinson_full_dataset.csv'. Обработано {len(df)} файлов.")
else:
    print("Не удалось обработать ни одного файла.")

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings

Датасет сохранен как 'parkinson_full_dataset.csv'. Обработано 72 файлов.
